In [ ]:
from generate_data import *
from utils import *
import numpy as np
from scipy.optimize import minimize
from numpy.linalg import norm
import warnings
warnings.filterwarnings("ignore")

global k, vk

In [ ]:
Ts = 0.1

N = 10
Nu = 8
Lambda = 5e-6

# 1 - population, 2 - tracking, 3 - hiv
process = 1

In [ ]:
est = 'good' # good / mid / bad
y_zad, fun, x, u, _, _, bds, umax, model = initialize(process, N, Nu, Ts, est)
model.print(precision=int(3 - np.log10(Ts)))

y_zad_N = np.vstack([y_zad, np.tile(y_zad[-1], (N, 1))])
kmax, vk = len(y_zad), 0

In [ ]:
def mpc_no(deltau, model=None, w=1):
    u[k:k+Nu] = (np.tril(np.ones((Nu, Nu)), 0) @ deltau.reshape(-1, 1) + u[k-1] * np.ones((Nu,1))).reshape(-1)
    u[k+Nu:k+N+1] = u[k+Nu-1]

    if model is not None:
        for p in range(N+1):
            x[k+p] = model.simulate(x[k+p-1], t=2, u=np.array([u[k+p-1]]))[-1] + vk
    else:
        for p in range(N+1):
            x[k+p] = fun(t=0, state=x[k+p-1], T=Ts, u=u[k+p-1])

    return norm((y_zad_N[k]*w).reshape(-1, 1)*np.ones((1, N)) - (x[k+1:k+N+1]*w).T, 2) + Lambda * norm(deltau, 2)

In [ ]:
state = x[0]
w = [1, 0]

for k in range(5, kmax):
    xmod =  model.simulate(state, t=2, u=np.array([u[k-1]]))[-1]
    state = fun(t=0, state=state, T=Ts, u=u[k-1])

    x[k, :] = state

    try:
        opt = minimize(
            mpc_no,
            args=(None, w),
            x0=np.zeros(Nu),
            method='SLSQP',
            bounds=[(-bds, bds)]*Nu,
            options={'maxiter': 100, 'ftol': 1e-6}
        )
        du = opt.x[0]
    except np.linalg.LinAlgError as e:
        print(f"Optimization failed due to: {e}")
        du = 0
    
    u[k] = u[k-1] + du
    u[k] = np.clip(u[k], -100, 100)
    vk = state - xmod

In [ ]:
plot(y_zad, x, u, w, save=False)